<a href="https://colab.research.google.com/github/BachVy/TRI-TUE-NHAN-TAO/blob/main/DOAN_TMDT_TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%%writefile tsp_result.py
from dataclasses import dataclass
from typing import List

@dataclass
class TSPResult:
    """Dataclass to hold TSP results."""
    min_cost: float
    path: List[int]

Writing tsp_result.py


In [9]:
%%writefile matrix_handler.py
import os
import numpy as np
from typing import Optional

class MatrixHandler:
    """OOP class for handling distance matrices (input, output, generation)."""

    def __init__(self, results_dir: str = "results"):
        self.results_dir = results_dir
        os.makedirs(self.results_dir, exist_ok=True)

    def read_from_file(self, file_path: str) -> np.ndarray:
        """Read symmetric distance matrix from TXT file."""
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File không tồn tại: {file_path}")

        matrix = []
        with open(file_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line:
                    row = [float(x.strip()) for x in line.replace(',', ' ').split() if x.strip()]
                    matrix.append(row)

        n = len(matrix)
        if n == 0:
            raise ValueError("File rỗng.")

        matrix = np.array(matrix)
        if matrix.shape != (n, n):
            raise ValueError(f"Ma trận không vuông {n}x{n}.")

        return matrix

    def create_random(self, n: int) -> np.ndarray:
        """Generate symmetric random distance matrix (diagonal=0)."""
        dist = np.random.randint(1, 101, size=(n, n)).astype(float)
        dist[np.diag_indices(n)] = 0
        dist = (dist + dist.T) / 2
        return dist

    def save_to_file(self, matrix: np.ndarray, filename: str) -> str:
        """Save matrix to TXT in results dir."""
        filepath = os.path.join(self.results_dir, filename)
        np.savetxt(filepath, matrix, fmt='%.2f', delimiter=' ')
        return filepath

Writing matrix_handler.py


In [15]:
%%writefile tsp_algorithm.py
import numpy as np
from typing import List
from tsp_result import TSPResult

class TSPAlgorithm:
    """OOP class for Nearest Neighbor Heuristic (Greedy) algorithm for TSP."""

    def __init__(self, verbose: bool = True):
        self.verbose = verbose

    def solve(self, dist_matrix: np.ndarray) -> TSPResult:
        """Solve TSP using Nearest Neighbor: Start from 0, greedily pick closest unvisited city."""
        n = len(dist_matrix)
        visited = [False] * n
        path = [0]  # Start from city 0
        visited[0] = True
        current_city = 0
        total_cost = 0.0

        if self.verbose:
            print("\n--- Nearest Neighbor từng bước ---")

        for step in range(1, n):
            min_dist = float('inf')
            next_city = -1

            # Tìm thành phố gần nhất chưa thăm
            for city in range(n):
                if not visited[city] and dist_matrix[current_city][city] < min_dist:
                    min_dist = dist_matrix[current_city][city]
                    next_city = city

            if next_city == -1:
                raise ValueError("Không thể tìm đường đi (ma trận không kết nối).")

            path.append(next_city)
            visited[next_city] = True
            total_cost += min_dist
            current_city = next_city

            if self.verbose:
                print(f"Bước {step}: Từ {current_city} đến {next_city} (chi phí {min_dist:.1f}, tích lũy {total_cost:.1f})")

        # Quay về 0
        return_cost = dist_matrix[current_city][0]
        total_cost += return_cost
        path.append(0)  # Close the tour (for visualization)

        if self.verbose:
            print(f"Quay về 0: +{return_cost:.1f} (tổng {total_cost:.1f})")
            print(f"Đường đi: {' -> '.join(map(str, path[:-1]))} -> 0")  # Exclude last 0 for print

        return TSPResult(min_cost=total_cost, path=path[:-1])  # Path without duplicate 0

Overwriting tsp_algorithm.py


In [16]:
%%writefile tsp_visualizer.py
import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from tsp_result import TSPResult

class TSPVisualizer:
    """OOP class for creating TSP animation."""

    def __init__(self, results_dir: str = "results"):
        self.results_dir = results_dir

    def create_animation(self, dist_matrix: np.ndarray, result: TSPResult) -> str:
        """Create and save GIF animation with cumulative cost."""
        n = len(dist_matrix)
        path = result.path
        G = nx.complete_graph(n)
        for i in range(n):
            for j in range(i + 1, n):
                G[i][j]['weight'] = dist_matrix[i][j]

        pos = nx.spring_layout(G, seed=42)
        fig, ax = plt.subplots(figsize=(10, 8))

        # Steps: cumulative edges
        steps = []
        for i in range(len(path) + 1):
            step_edges = [(path[j], path[(j + 1) % len(path)]) for j in range(i)] if i > 0 else []
            steps.append(step_edges)

        def init():
            nx.draw_networkx_edges(G, pos, ax=ax, alpha=0.2, width=0.5)
            nx.draw_networkx_nodes(G, pos, ax=ax, node_color='lightblue', node_size=500)
            nx.draw_networkx_labels(G, pos, ax=ax, labels={i: str(i) for i in range(n)}, font_size=12, font_weight='bold')
            ax.set_title("TSP: Bắt đầu từ 0 (Chi phí: 0.0)")
            return ax

        def animate(frame):
            ax.clear()
            nx.draw_networkx_edges(G, pos, ax=ax, alpha=0.2, width=0.5)
            nx.draw_networkx_nodes(G, pos, ax=ax, node_color='lightblue', node_size=500)
            nx.draw_networkx_labels(G, pos, ax=ax, labels={i: str(i) for i in range(n)}, font_size=12, font_weight='bold')

            current_edges = steps[frame]
            if current_edges:
                nx.draw_networkx_edges(G, pos, ax=ax, edgelist=current_edges, edge_color='red', width=2, arrows=True, arrowsize=20)

            # Cumulative cost up to frame
            current_cost = 0.0
            if frame > 0:
                for j in range(frame):
                    from_city = path[j]
                    to_city = path[(j + 1) % len(path)]
                    current_cost += dist_matrix[from_city, to_city]

            current_path_str = ' -> '.join(map(str, path[:frame + 1]))
            if frame == len(path):
                current_path_str += ' -> 0'

            title = f"Bước {frame}: {current_path_str} (Chi phí tích lũy: {current_cost:.1f})"
            ax.set_title(title)
            return ax

        anim = FuncAnimation(fig, animate, init_func=init, frames=len(steps), interval=1500, blit=False, repeat=True)

        # Save GIF to results dir
        gif_path = os.path.join(self.results_dir, 'tsp_animation.gif')
        anim.save(gif_path, writer='pillow', fps=0.67)
        plt.close(fig)  # Close to free memory

        return gif_path

Overwriting tsp_visualizer.py


In [ ]:

import os
from matrix_handler import MatrixHandler
from tsp_algorithm import TSPAlgorithm
from tsp_visualizer import TSPVisualizer
from tsp_result import TSPResult

def main():
    """Main function to run TSP application."""
    results_dir = "results"
    os.makedirs(results_dir, exist_ok=True)

    # Input
    file_path = input("Nhập đường dẫn file ma trận khoảng cách (TXT) hoặc 'random': ").strip()

    matrix_handler = MatrixHandler(results_dir)
    algo = TSPAlgorithm(verbose=True)
    visualizer = TSPVisualizer(results_dir)

    if file_path.lower() == 'random':
        n = int(input("Nhập số thành phố (n, khuyến nghị ≤15): "))
        dist_matrix = matrix_handler.create_random(n)
        saved_matrix_path = matrix_handler.save_to_file(dist_matrix, 'random_dist_matrix.txt')
        print(f"Tạo và lưu ma trận vào '{saved_matrix_path}'.")
    else:
        try:
            dist_matrix = matrix_handler.read_from_file(file_path)
            n = len(dist_matrix)
            print(f"Đã đọc ma trận từ '{file_path}'. Số thành phố: {n}")
            saved_matrix_path = None
        except Exception as e:
            print(f"Lỗi: {e}. Fallback random.")
            n = int(input("Nhập số thành phố (n): "))
            dist_matrix = matrix_handler.create_random(n)
            saved_matrix_path = matrix_handler.save_to_file(dist_matrix, 'random_dist_matrix.txt')
            print(f"Tạo và lưu ma trận fallback vào '{saved_matrix_path}'.")

    # Run TSP
    result: TSPResult = algo.solve(dist_matrix)


    print(f"\n=== KẾT QUẢ TSP ===")
    print(f"Chi phí min: {result.min_cost:.2f}")
    print(f"Đường đi: {' -> '.join(map(str, result.path))} -> 0")

    # Create and save animation
    print("\nĐang tạo GIF animation...")
    gif_path = visualizer.create_animation(dist_matrix, result)
    print(f"Animation lưu thành '{gif_path}'")

if __name__ == "__main__":
    main()